In [11]:
#Import the libraries
import pickle
import time
import pandas as pd
import requests

In [10]:
headers = {"User-agent":"secret_agent"}

In [3]:
#Check that the url  is working

url_puns = "https://www.reddit.com/r/puns.json"
url_politics = "https://www.reddit.com/r/politics.json"

res_puns = requests.get(url_puns, headers = headers)
res_politics = requests.get(url_politics, headers = headers)

print(res_puns.status_code)
print(res_politics.status_code)

200
200


In [4]:
#Web scraps the post titles from the Puns Subreddit Thread through Reddit's API and add them to a list
puns_title = []
after = None
for i in range(38):
    if after == None:
        params = {}
    else:
        params = {"after": after}
    url = "https://www.reddit.com/r/puns.json"
    res = requests.get(url, params=params, headers=headers)
    if res.status_code == 200:
        the_json = res.json()
        for j in range(25):
            puns_title.append(the_json["data"]["children"][j]["data"]["title"])
            after = the_json["data"]["after"]
    else:
        print(res.status_code)
        break
    time.sleep(1)

In [5]:
#Web scraps the post titles from the Politics Subreddit Thread through Reddit's API and add them to a list
politics_title = []
after = None
for i in range(35):
    if after == None:
        params = {}
    else:
        params = {"after": after}
    url = "https://www.reddit.com/r/politics.json"
    res = requests.get(url, params=params, headers=headers)
    if res.status_code == 200:
        the_json = res.json()
        for j in range(25):
            politics_title.append(the_json["data"]["children"][j]["data"]["title"])
            after = the_json["data"]["after"]
    else:
        print(res.status_code)
        break
    time.sleep(1)

In [6]:
#Check the length of unique entries
print(len(set(puns_title)))
print(len(set(politics_title)))

939
870


In [7]:
#Convert the list to a set and then to Pandas Dataframe, convert all strings to lowercase and remove special characters
puns_title2 = pd.DataFrame(set(puns_title)).rename(columns={0:"title"}).title.str.lower().str.replace('[^\w\s]','').to_frame()
politics_title2 = pd.DataFrame(set(politics_title)).rename(columns={0:"title"}).title.str.lower().str.replace('[^\w\s]','').to_frame()

#Differentiate between the two threads
puns_title2["target"] = 1
politics_title2["target"] = 0

#Combine the dataframes together
df_title = pd.concat([puns_title2, politics_title2]).reset_index().drop(columns=["index"])

In [8]:
#Export the data to csv
df_title.to_csv('../data/df_title.csv')

In [16]:
#Export list to txt
with open("../data/puns_title.txt", "wb") as fp:
    pickle.dump(puns_title, fp)
    
with open("../data/politics_title.txt", "wb") as fp:
    pickle.dump(politics_title, fp)